In [1]:
import os
import re
import requests
import time

import pandas as pd
import numpy as np

from tqdm import tqdm_notebook
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings(action = 'ignore')

In [64]:
# Open API 수집을 위한 ServiceKey 불러오기
with open('ServiceKey.txt', 'r') as file:
    service_key = file.read()

In [66]:
# 시도명을 직접 입력하여 정보를 수집해야하므로, 시도명을 나타내는 리스트 객체 생성
sido_list = ['인천', '강원', '경남', '경북', '부산', '울산', '전남', '전북', '충남', '제주']

# 해수욕장정보 서비스 Open API URL
# 모든 시도명에 해수욕장이 100개 이하이므로, 모든 자료를 불러오기 위해 'numOfRows=100' 으로 지정해주었다.
URL = 'http://apis.data.go.kr/1192000/service/OceansBeachInfoService1/getOceansBeachInfo1?&numOfRows=100&ServiceKey={}'.format(service_key)

# 시도명, 명칭, 경도, 위도 데이터를 수집하기 위한 리스트 객체 생성
시도명 = []
명칭 = []
경도 = []
위도 = []

# 시도명을 반복하며 데이터 수집
for name in tqdm_notebook(sido_list, desc = '해수욕장정보 서비스 데이터 수집'):
    
    URL = URL + '&SIDO_NM={}'.format(name)
    
    html = requests.get(URL)
    soup = BeautifulSoup(html.content, 'html.parser')
    
    # 각 정보에 해당되는 태그를 찾아 리스트에 저장
    시도명.append([value.get_text().strip() for value in soup.find_all('sidonm')])
    명칭.append([value.get_text().strip() for value in soup.find_all('stanm')])
    경도.append([float(value.get_text().strip()) for value in soup.find_all('lon')])
    위도.append([float(value.get_text().strip()) for value in soup.find_all('lat')])
    
    time.sleep(5)

해수욕장정보 서비스 데이터 수집:   0%|          | 0/10 [00:00<?, ?it/s]

In [67]:
# 수집된 정보들이 2차원 리스트 구조이기 때문에 1차원 구조로 변경해주어야 한다.
시도명 = sum(시도명, [])
명칭 = sum(명칭, [])
경도 = sum(경도, [])
위도 = sum(위도, [])

# 수집한 데이터 일부 확인
print(시도명[:5], '\n')
print(명칭[:5], '\n')
print(경도[:5], '\n')
print(위도[:5])

['인천', '인천', '인천', '인천', '인천'] 

['대빈창', '동막', '민머루', '조개골', '구리동'] 

[126.22844, 126.458791, 126.333573, 126.199815, 125.685328] 

[37.646383, 37.592434, 37.651038, 37.658272, 37.667963]


In [68]:
# 수집한 데이터를 DataFrame으로 저장
beach_information = pd.DataFrame({'시도명' : 시도명, '명칭' : 명칭, '경도' : 경도, '위도' : 위도})

# 데이터 일부 확인
display(beach_information.head())

,시도명,명칭,경도,위도
0,인천,대빈창,126.228440,37.646383
1,인천,동막,126.458791,37.592434
2,인천,민머루,126.333573,37.651038
3,인천,조개골,126.199815,37.658272
4,인천,구리동,125.685328,37.667963


```python
beach_information.to_csv('./Data/해수욕장정보 서비스.csv', encoding = 'CP949', index = False)
```